In [ ]:
#| default_exp processing.postproc

# Postprocessing

> Smoothing, combining etc.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from drone_detector.imports import *
from drone_detector.utils import *

from skimage.morphology import erosion, dilation
from scipy.ndimage.morphology import binary_fill_holes

## Non-maximum suppression

First the commonly used NMS with bounding boxes, that prioritizes either confidence score (default) or bounding box area.

In [ ]:
#| export
 
# Malisiewicz et al.
def non_max_suppression_fast(boxes, scores, overlap_thresh:float, sort_criterion:str='score'):
    "Possibility to sort boxes by score (default) or area"
    
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # sort prediction by scores, 

    # initialize the list of picked indexes    
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    if sort_criterion == 'score':
        idxs = np.argsort(scores)
    elif sort_criterion == 'area':
        idxs = np.argsort(area)
    else:
        print('Unknown sort criteria, reverting to "score"')
        idxs = np.argsort(scores)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
                         np.where(overlap > overlap_thresh)[0])))
 
    # return indices for selected bounding boxes
    return pick
    #return boxes[pick].astype("int")

Non-max suppression can in theory be applied also on polygons, but it hasn't been used in any publications as far as I know.

If `non_max_suppression_poly` is used to eliminate polygons, threshold might need to be smaller than typical value of 0.7 that is used.

In [ ]:
#| export

from drone_detector.metrics import poly_IoU

In [ ]:
#| export

def non_max_suppression_poly(geoms, scores, overlap_thresh:float, sort_criterion:str='score'):
    "Possibility to sort geoms by score (default) or area"
    
    # if there are no geoms, return an empty list
    if len(geoms) == 0:
        return []
 
 
    # sort prediction by scores, 

    # initialize the list of picked indexes    
    pick = []
 
    # compute the area of the bounding geoms and sort the bounding
    # geoms by the bottom-right y-coordinate of the bounding box
    area = np.array([geom.area for geom in geoms])
    if sort_criterion == 'score':
        idxs = np.argsort(scores)
    elif sort_criterion == 'area':
        idxs = np.argsort(area)
    else:
        print('Unknown sort criteria, reverting to "score"')
        idxs = np.argsort(scores)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # compute the ratio of overlap with all other polygons
        overlap = np.array([poly_IoU(geoms[i], geoms[k]) for k in idxs[:last]])
        # delete all indexes from the index list that have
        # overlap larger than overlap_thresh
        idxs = np.delete(idxs, np.concatenate(([last],
                         np.where(overlap > overlap_thresh)[0])))
         
    # return indices for selected bounding geoms
    return pick

Some utils to run above functions to `GeoDataFrames`

In [ ]:
#| export

def do_nms(gdf:gpd.GeoDataFrame, nms_thresh=0.7, crit='score'):
    gdf = gdf.copy()
    np_bboxes = np.array([b.bounds for b in gdf.geometry])
    scores = gdf.score.values
    idxs = non_max_suppression_fast(np_bboxes, scores, nms_thresh, crit)
    gdf = gdf.iloc[idxs]
    return gdf

def do_poly_nms(gdf:gpd.GeoDataFrame, nms_thresh=0.1, crit='score'):
    gdf = gdf.copy()
    scores = gdf.score.values
    idxs = non_max_suppression_poly(gdf.geometry.values, scores, nms_thresh, crit)
    gdf = gdf.iloc[idxs]
    return gdf

def do_min_rot_rectangle_nms(gdf:gpd.GeoDataFrame, nms_thresh=0.7, crit='score'):
    gdf = gdf.copy()
    scores = gdf.score.values
    boxes = np.array([g.minimum_rotated_rectangle for g in gdf.geometry.values])
    idxs = non_max_suppression_poly(boxes, scores, nms_thresh, crit)
    gdf = gdf.iloc[idxs]
    return gdf

# Weighted boxes fusion

Originally presented by [Solovyev et al (2021)](https://arxiv.org/abs/1910.13302), and available in [https://github.com/ZFTurbo/Weighted-Boxes-Fusion]. Code presented here is modified to keep track of original bounding boxes for mask fusion, and due to numpy version requirements we do not use numba here.

As WBF expects normalized coordinates, first some helpers to normalize and denormalize geocoordinates. 

In [ ]:
#| export

def normalize_bbox_coords(tot_bounds, bboxes):
    xmin = tot_bounds[0]
    ymin = tot_bounds[1]
    width = tot_bounds[2] - tot_bounds[0]
    height = tot_bounds[3] - tot_bounds[1]
    
    norm_bboxes = [((b[0]-xmin)/(width), 
                    (b[1]-ymin)/(height),
                    (b[2]-xmin)/(width),
                    (b[3]-ymin)/(height))
                  for b in bboxes]
    return norm_bboxes


def denormalize_bbox_coords(tot_bounds, bboxes):
    xmin = tot_bounds[0]
    ymin = tot_bounds[1]
    width = tot_bounds[2] - tot_bounds[0]
    height = tot_bounds[3] - tot_bounds[1]
    
    norm_bboxes = [((b[0]*width+xmin), 
                    (b[1]*height+ymin),
                    (b[2]*width+xmin),
                    (b[3]*height+ymin))
                  for b in bboxes]
    return norm_bboxes


In [ ]:
#|export 
import warnings

In [ ]:
#| export

def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


def prefilter_boxes(boxes, scores, labels, weights, thr):
    # Create dict with boxes stored by its label
    new_boxes = dict()

    for t in range(len(boxes)):

        if len(boxes[t]) != len(scores[t]):
            print('Error. Length of boxes arrays not equal to length of scores array: {} != {}'.format(len(boxes[t]), len(scores[t])))
            sys.exit()

        if len(boxes[t]) != len(labels[t]):
            print('Error. Length of boxes arrays not equal to length of labels array: {} != {}'.format(len(boxes[t]), len(labels[t])))
            sys.exit()

        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = int(labels[t][j])
            box_part = boxes[t][j]
            x1 = max(float(box_part[0]), 0.)
            y1 = max(float(box_part[1]), 0.)
            x2 = max(float(box_part[2]), 0.)
            y2 = max(float(box_part[3]), 0.)

            # Box data checks
            if x2 < x1:
                warnings.warn('X2 < X1 value in box. Swap them.')
                x1, x2 = x2, x1
            if y2 < y1:
                warnings.warn('Y2 < Y1 value in box. Swap them.')
                y1, y2 = y2, y1
            if x1 > 1:
                warnings.warn('X1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x1 = 1
            if x2 > 1:
                warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x2 = 1
            if y1 > 1:
                warnings.warn('Y1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y1 = 1
            if y2 > 1:
                warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y2 = 1
            if (x2 - x1) * (y2 - y1) == 0.0:
                warnings.warn("Zero area box skipped: {}.".format(box_part))
                continue

            # [label, score, weight, model index, x1, y1, x2, y2]
            b = [int(label), float(score) * weights[t], weights[t], t, x1, y1, x2, y2]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        current_boxes = np.array(new_boxes[k])
        new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]

    return new_boxes


def get_weighted_box(boxes, conf_type='avg'):
    """
    Create weighted box for set of boxes
    :param boxes: set of boxes to fuse
    :param conf_type: type of confidence one of 'avg' or 'max'
    :return: weighted box (label, score, weight, x1, y1, x2, y2)
    """

    box = np.zeros(8, dtype=np.float32)
    conf = 0
    conf_list = []
    w = 0
    for b in boxes:
        box[4:] += (b[1] * b[4:])
        conf += b[1]
        conf_list.append(b[1])
        w += b[2]
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    elif conf_type in ['box_and_model_avg', 'absent_model_aware_avg']:
        box[1] = conf / len(boxes)
    box[2] = w
    box[3] = -1 # model index field is retained for consistensy but is not used.
    box[4:] /= conf
    return box


def find_matching_box_quickly(boxes_list, new_box, match_iou):
    """ Reimplementation of find_matching_box with numpy instead of loops. Gives significant speed up for larger arrays
        (~100x). This was previously the bottleneck since the function is called for every entry in the array.
    """
    def bb_iou_array(boxes, new_box):
        # bb interesection over union
        xA = np.maximum(boxes[:, 0], new_box[0])
        yA = np.maximum(boxes[:, 1], new_box[1])
        xB = np.minimum(boxes[:, 2], new_box[2])
        yB = np.minimum(boxes[:, 3], new_box[3])

        interArea = np.maximum(xB - xA, 0) * np.maximum(yB - yA, 0)

        # compute the area of both the prediction and ground-truth rectangles
        boxAArea = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        boxBArea = (new_box[2] - new_box[0]) * (new_box[3] - new_box[1])

        iou = interArea / (boxAArea + boxBArea - interArea)

        return iou

    if boxes_list.shape[0] == 0:
        return -1, match_iou

    # boxes = np.array(boxes_list)
    boxes = boxes_list

    ious = bb_iou_array(boxes[:, 4:], new_box[4:])

    ious[boxes[:, 0] != new_box[0]] = -1

    best_idx = np.argmax(ious)
    best_iou = ious[best_idx]

    if best_iou <= match_iou:
        best_iou = match_iou
        best_idx = -1

    return best_idx, best_iou


def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):
    '''
    :param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1]
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value, 'box_and_model_avg': box and model wise hybrid weighted average, 'absent_model_aware_avg': weighted average that takes into account the absent model.
    :param allows_overflow: false if we want confidence score not exceed 1.0
    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
    :return: scores: confidence scores
    :return: labels: boxes labels
    :return: originals: original boxes
    '''

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max', 'box_and_model_avg', 'absent_model_aware_avg']:
        print('Unknown conf_type: {}. Must be "avg", "max" or "box_and_model_avg", or "absent_model_aware_avg"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    original_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = np.empty((0,8))
        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box_quickly(weighted_boxes, boxes[j], iou_thr)

            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes = np.vstack((weighted_boxes, boxes[j].copy()))
        original_boxes.append(new_boxes)
        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            clustered_boxes = np.array(new_boxes[i])
            if conf_type == 'box_and_model_avg':
                # weighted average for boxes
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / weighted_boxes[i, 2]
                # identify unique model index by model index column
                _, idx = np.unique(clustered_boxes[:, 3], return_index=True)
                # rescale by unique model weights
                weighted_boxes[i, 1] = weighted_boxes[i, 1] *  clustered_boxes[idx, 2].sum() / weights.sum()
            elif conf_type == 'absent_model_aware_avg':
                # get unique model index in the cluster
                models = np.unique(clustered_boxes[:, 3]).astype(int)
                # create a mask to get unused model weights
                mask = np.ones(len(weights), dtype=bool)
                mask[models] = False
                # absent model aware weighted average
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / (weighted_boxes[i, 2] + weights[mask].sum())
            elif conf_type == 'max':
                weighted_boxes[i, 1] = weighted_boxes[i, 1] / weights.max()
            elif not allows_overflow:
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * min(len(weights), len(clustered_boxes)) / weights.sum()
            else:
                weighted_boxes[i, 1] = weighted_boxes[i, 1] * len(clustered_boxes) / weights.sum()
        overall_boxes.append(weighted_boxes)
    original_boxes = [item for sublist in original_boxes for item in sublist]
    overall_boxes = np.concatenate(overall_boxes, axis=0)
    sidx = overall_boxes[:, 1].argsort()
    overall_boxes = overall_boxes[sidx[::-1]]
    boxes = overall_boxes[:, 4:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    # sort originals accoring to wbf
    #original_boxes = original_boxes[0]
    wbfo = [original_boxes[i] for i in sidx[::-1]]
    return boxes, scores, labels, wbfo

In [ ]:
#| export

def do_wbf(gdf:gpd.GeoDataFrame, iou_thr=0.55, skip_box_thr=0.5):
    """Run weighted_boxes_fusion and returns a gpd.GeoDataFrame where geometries are replaced by new bounding boxes.
    Do not use with instance segmentation data unless you want to replace your results with bounding boxes
    """
    np_bboxes = [b.bounds for b in gdf.geometry]
    np_bboxes = normalize_bbox_coords(gdf.total_bounds, np_bboxes)
    np_bboxes = [[b for b in np_bboxes]]
    scores = [[v for v in gdf.score.values]]
    labels = [[l for l in gdf.label.values]]
    wbf_boxes, wbf_scores, wbf_labels, _ = weighted_boxes_fusion(np_bboxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    wbf_gdf = gpd.GeoDataFrame()
    wbf_gdf['label'] = wbf_labels
    wbf_gdf['score'] = wbf_scores
    wbf_boxes = denormalize_bbox_coords(gdf.total_bounds, wbf_boxes)
    wbf_gdf['geometry'] = [box(*bbox) for bbox in wbf_boxes]
    wbf_gdf.crs = gdf.crs
    return wbf_gdf


In [ ]:
#| export

def do_wsf(gdf:gpd.GeoDataFrame, iou_thr=0.55, skip_box_thr=0.5):
    np_bboxes = [b.bounds for b in gdf.geometry]
    np_bboxes = normalize_bbox_coords(gdf.total_bounds, np_bboxes)
    np_bboxes = [[b for b in np_bboxes]]
    scores = [[v for v in gdf.score.values]]
    labels = [[l for l in gdf.label.values]]
    wbf_boxes, wbf_scores, wbf_labels, originals = weighted_boxes_fusion(np_bboxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    wbf_gdf = gpd.GeoDataFrame()
    wbf_gdf['label'] = wbf_labels
    wbf_gdf['score'] = wbf_scores
    wbf_boxes = denormalize_bbox_coords(gdf.total_bounds, wbf_boxes)
    wbf_gdf['geometry'] = [box(*bbox) for bbox in wbf_boxes]
    wbf_gdf.crs = gdf.crs
    
    wbf_scores = []
    wbf_labels = []
    wbf_masks = []
    
    for i, wbox in tqdm(enumerate(wbf_gdf.itertuples())):
        wbf_scores.append(wbox.score)
        wbf_labels.append(wbox.label)
        orig_bboxes = [bbox[4:] for bbox in originals[i]]
        orig_bboxes = denormalize_bbox_coords(gdf.total_bounds, orig_bboxes)
        orig_bboxes = [box(*bounds) for bounds in orig_bboxes]
        orig_masks = [m for m in gdf.geometry if box(*m.bounds) in orig_bboxes]
        mask = shapely.ops.unary_union(orig_masks)
        mask = mask.intersection(wbox.geometry)
        wbf_masks.append(mask)
    
    mask_gdf = gpd.GeoDataFrame()
    mask_gdf['label'] = wbf_labels
    mask_gdf['score'] = wbf_scores
    mask_gdf['geometry'] = wbf_masks
    mask_gdf.crs = gdf.crs
    
    return mask_gdf

## Smoothing and filling holes

Below functions are run before converting IceVision preds to COCO or shapefile format.

In [ ]:
#| export

def fill_holes(preds:list) -> list:
    "Run `binary_fill_holes` to predicted binary masks"
    for i, p in tqdm(enumerate(preds)):
        for j in rangeof(p.pred.detection.label_ids):
            p_mask = p.pred.detection.mask_array.to_mask(p.height, p.width).data[j]
            p.pred.detection.mask_array.data[j] = binary_fill_holes(p_mask).astype(np.int8)
    return preds
    
def dilate_erode(preds:list) -> list:
    "Run dilation followed by erosion in order to smooth masks"
    for i, p in tqdm(enumerate(preds)):
        for j in rangeof(p.pred.detection.label_ids):
            p_mask = p.pred.detection.mask_array.to_mask(p.height, p.width).data[j]
            p.pred.detection.mask_array.data[j] = erosion(dilation(p_mask))
    return preds